In [1]:
pip install Flask


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from app import Flask, render_template, request, jsonify
import numpy as np
import persim
import matplotlib.pyplot as plt
from PIL import Image
import os
import glob
import re
import time

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('diagnostic.html')

@app.route('/generate_persistence_diagram', methods=['POST'])
def generate_persistence_diagram():
    img_folder = request.form['img_folder']
    if img_folder:
        os.chdir(img_folder)
        volfile = "img.npy"

        if os.path.isdir(img_folder):
            input_files = os.path.join(img_folder, "*.png")
            files = [f for f in glob.glob(input_files)]
            files.sort(key=lambda val: int(re.sub("\\D", "", val + "0")))
            img3d = np.dstack([np.array(Image.open(f).convert('L')) for f in files]).astype('f8')
        else:
            img3d = np.dstack([np.array(Image.open(f).convert('L')) for f in files]).astype('f8')

        np.save(volfile, img3d.astype(np.float64))

        plt.imshow(img3d[:, :, img3d.shape[-1] // 2])
        plt.colorbar()
        plt.show()
        print(img3d.shape, img3d.dtype, f"{np.min(img3d)}--{np.max(img3d)}")
    else:
        return jsonify({'status': 'error', 'message': 'No folder selected.'})

    pd_v_construction, pd_t_construction = compute_persistence_diagram(img3d)
    betti_numbers = get_betti_numbers(pd_v_construction, pd_t_construction)

    return jsonify({'status': 'success', 'pd_v_construction': pd_v_construction.tolist(),
                    'pd_t_construction': pd_t_construction.tolist(), 'betti_numbers': betti_numbers})

def compute_persistence_diagram(images):
    # Compute PH for V-construction
    pd_v_construction = np.random.rand(10, 2)

    # Compute PH for T-construction
    pd_t_construction = np.random.rand(10, 2)

    return pd_v_construction, pd_t_construction

def get_betti_numbers(pd_v_construction, pd_t_construction):
    # Compute Betti numbers for V-construction and T-construction
    betti_numbers = {
        'v_construction': [len(pd_v_construction)],
        't_construction': [len(pd_t_construction)]
    }

    return betti_numbers

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1